In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
#from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import sqlite3
from random import randint

c:\Users\chris\miniconda3\envs\LSTF_Linear\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


I propose that the worst performance for ACI will be when the distribution changes at the same intervals as the lookback window as at that point there is a gaurantee that the quantile will be estimated with data which is not relevant to the now current distribution.

In [2]:
dist_period =100

In [3]:
# This creates a TS, where the distribution changes at different points deppending on a provided list of distribution shifts.
def Create_TimeSeries(dist_shifts):
    final = np.array([])
    
    for i in range(len(dist_shifts)):
        Y = abs(np.random.normal(dist_shifts[i][0], dist_shifts[i][1], dist_period))
        final = np.concatenate((final, Y))
    
    return final + 0.5*np.roll(final, 1)

# Generating the data.

In [4]:


p = 0.5
pdist, steps = [1-p, p], [-1, 1]

datapoints = 200

train_Output = []

# These are the distribution shifts that will be applied to the time series
dist_shift = [(randint(-10, 10), randint(1, 10)) for _ in range(10)]
#dist_shift = [(0,1), (2,0)]
#dist_shift = [((-1**x) * 100, 100) if x%5==0 else (0, 0) for x in range(30)]

for _ in range(datapoints):
    X =  np.random.choice(steps, size=len(dist_shift)*dist_period, p=pdist)
    Y = Create_TimeSeries(dist_shift)
    T = np.cumsum(X*Y)
    
    input_data = T[:-1]
    labels_data = T[1:]

    train_Output.append((input_data, labels_data))


In [5]:
# This function predicts a range of Y_t at the alpha level. By using alpha_t.
def C_t(alpha_t, scores, sigma_t, t, interval=50):
    alpha_t = min(1, max(0, alpha_t))
    Q = np.quantile(scores[t-interval:t], 1 - alpha_t)
    positve_v = (sigma_t) + (abs(sigma_t) * Q)
    negative_v = (sigma_t) - (abs(sigma_t) * Q)
    return negative_v, positve_v

# This function returns 1 if the prediction lies in the interval, 0 otherwise.
def err_t(Y_t, C_t):
    if C_t[0] < Y_t < C_t[1]:
        return 0
    else:
        return 1

In [6]:
def ACI(coverage_target: float, timeseries_data: tuple, gamma: float, custom_interval: int) -> dict:
    xpred, y = timeseries_data
    alpha_t_list = [coverage_target]

    All_scores = (abs(y - xpred))/abs(xpred)

    err_t_list = []
    conformal_sets_list = []
    
    for i in range(custom_interval+1, len(All_scores)):
        Coverage_t = C_t(alpha_t_list[-1], All_scores, xpred[i], i, custom_interval)
        conformal_sets_list.append(Coverage_t)

        error_t = err_t(y[i], Coverage_t)
        err_t_list.append(error_t)

        alpha_t = min(max(alpha_t_list[-1] + (gamma * (coverage_target - error_t)), 0), 1)
        alpha_t_list.append(alpha_t)

    # Calculating different averages
    realised_interval_coverage = 1 - pd.Series(err_t_list).rolling(custom_interval).mean().mean()
    average_prediction_interval = np.mean([abs(x[1] - x[0]) for x in conformal_sets_list])

    return {
        'model': 'ACI',
        'coverage_target': coverage_target,
        'gamma': gamma,
        'realised_interval_coverage': realised_interval_coverage,
        'alpha_t_list': alpha_t_list,
        'average_prediction_interval': average_prediction_interval,
        'conformal_sets': conformal_sets_list,
        'error_t_list': err_t_list, 
        'interval_size': custom_interval
    }

In [7]:
def scale_C_t(scale, alpha_t, scores, sigma_t, t, interval=50):
    alpha_t = min(1, max(0, alpha_t))
    Q = np.quantile(scores[t-interval:t], 1 - alpha_t)
    positve_v = (sigma_t) + scale *(abs(sigma_t) * Q)
    negative_v = (sigma_t) - scale *(abs(sigma_t) * Q)
    return negative_v, positve_v

def scaled_ACI(scale: float, coverage_target: float, timeseries_data: tuple, gamma: float, custom_interval: int) -> dict:
    xpred, y = timeseries_data
    alpha_t_list = [coverage_target]

    All_scores = (abs(y - xpred))/abs(xpred)

    err_t_list = []
    conformal_sets_list = []
    
    for i in range(custom_interval+1, len(All_scores)):
        Coverage_t = scale_C_t(scale, alpha_t_list[-1], All_scores, xpred[i], i, custom_interval)
        conformal_sets_list.append(Coverage_t)

        error_t = err_t(y[i], Coverage_t)
        err_t_list.append(error_t)

        alpha_t = min(max(alpha_t_list[-1] + (gamma * (coverage_target - error_t)), 0), 1)
        alpha_t_list.append(alpha_t)

    # Calculating different averages
    realised_interval_coverage = 1 - pd.Series(err_t_list).rolling(custom_interval).mean().mean()
    average_prediction_interval = np.mean([abs(x[1] - x[0]) for x in conformal_sets_list])

    return {
        'model': 'scaled_ACI',
        'coverage_target': coverage_target,
        'gamma': gamma,
        'realised_interval_coverage': realised_interval_coverage,
        'alpha_t_list': alpha_t_list,
        'average_prediction_interval': average_prediction_interval,
        'conformal_sets': conformal_sets_list,
        'error_t_list': err_t_list, 
        'interval_size': custom_interval
    }

In [8]:
from ConformalMethods import ACP_plots

records = [[], []]
for data in train_Output:
    scaled = scaled_ACI(1.05, 0.3, data, 0.1, 15)
    non_scaled = ACI(0.3, data, 0.5, 10)
    records[0].append(scaled['realised_interval_coverage'])
    records[1].append(non_scaled['realised_interval_coverage'])
    print('scaled', np.mean(records[0]), 'non-scaled', np.mean(records[1]))
    ACP_plots.compare_many([scaled, non_scaled])

scaled 0.6996216030271758 non-scaled 0.6512768130745659


ValueError: Invalid RGBA argument: 'o'

<Figure size 1440x720 with 4 Axes>

scaled 0.6985896112831098 non-scaled 0.6519918283963227


ValueError: Invalid RGBA argument: 'o'

...

scaled 0.697582637520718 non-scaled 0.6499210697372086


ValueError: Invalid RGBA argument: 'o'

<Figure size 1440x720 with 4 Axes>

scaled 0.6976294102690658 non-scaled 0.6497979304525469


KeyboardInterrupt: 

In [ ]:
import inspect
import numpy as np 

print(inspect.getsource(np.quantile))

@array_function_dispatch(_quantile_dispatcher)
def quantile(a, q, axis=None, out=None,
             overwrite_input=False, interpolation='linear', keepdims=False):
    """
    Compute the q-th quantile of the data along the specified axis.

    .. versionadded:: 1.15.0

    Parameters
    ----------
    a : array_like
        Input array or object that can be converted to an array.
    q : array_like of float
        Quantile or sequence of quantiles to compute, which must be between
        0 and 1 inclusive.
    axis : {int, tuple of int, None}, optional
        Axis or axes along which the quantiles are computed. The
        default is to compute the quantile(s) along a flattened
        version of the array.
    out : ndarray, optional
        Alternative output array in which to place the result. It must
        have the same shape and buffer length as the expected output,
        but the type (of the output) will be cast if necessary.
    overwrite_input : bool, optional
        

Observed that sometimes. prob due to beng unluck we have that one weight gets over assigned probabilty and then it gets chosen almost every time which can cause a terrible result which throws of the entire average. Am also noticing that the variamce is of dtaci is consittently lower which supports the hypothesis that it is not correctig its mistakes.

Also the fact that there is chanve involved means that sometimes a clearly bad weight is chosen by chance.